In [13]:
import os
import numpy as np
import pandas as p
import zipfile
import fileinput

In [14]:
#directory path
path = os.getcwd() + "/"
path_t = path + "source_tables/"
path_b = path_t + "built/"

In [15]:
#1 for activating, 0 not

In [16]:
#unzip the tables - *must run at first use*
unzip = 0

In [32]:
#*must run at first use*
#prepare events
prep_events = 0

#prepare topics and categories
prep_topics_categories = 0

#prepare promoted content adding topics and categories
prep_promoted_content = 1

In [33]:
#optional, these files are already in the zips
#build all the ctr tables
build_ctr = 0

#build the timestamp table with offsets by geographic location
build_time = 0
#import and use shapely
shapley_f = 1

In [34]:
#unzip
if unzip:
    zip_ref = zipfile.ZipFile(path_t + "tables.zip", 'r')
    zip_ref.extractall(path_t)
    zip_ref.close()
    zip_ref = zipfile.ZipFile(path_b + "built_tables.zip", 'r')
    zip_ref.extractall(path_b)
    zip_ref.close()
    print "done unzipping"
    
    
if build_ctr:
    #create all the CTR tables to load later, once created no need to recreate
    ctr_features = path + "ctr_features.py"
    %run $ctr_features
    
    
if build_time:
    if shapley_f:
        for line in fileinput.input([path + "timezone.py", path + "notebooks/timezone.ipynb"], inplace = 1): 
            print line.replace("shapely_f = 0", "shapely_f = 1").rstrip()
    else:
        for line in fileinput.input([path + "timezone.py", path + "notebooks/timezone.ipynb"], inplace = 1): 
            print line.replace("shapely_f = 1", "shapely_f = 0").rstrip()
    #create the table of timestamps corrected by the offset got by geographic location, once created no need to recreate
    timezone = path + "timezone.py"
    %run $timezone

    
if prep_events:
    events = p.read_csv(path_b + "events.csv", usecols = ["display_id", "document_id", "platform"], dtype={"display_id":int, "document_id":int, "platform":str})
    
    #cast all platform to int, 5 missing values, changed to median
    events.loc[events['platform'] == '\\N', 'platform'] = events['platform'][events['platform'] != '\\N'].median()
    events['platform'] = events['platform'].apply(int)
    
    #prepare one-hot for platform
    platform_dummies = p.get_dummies(events['platform'],prefix="plat").astype(int)
    events = events.drop('platform',axis = 1).join(platform_dummies)
    del platform_dummies
    
    events.to_csv(path_b + 'events_prep.csv', index=False)
    print "events_prep.csv created in path/source_tables/built/ directory"
    

if prep_topics_categories:
    documents_topics = p.read_csv(path_t + "documents_topics.csv", dtype = {"document_id":int, "topic_id":int, "confidence_level":float})
    documents_categories = p.read_csv(path_t + "documents_categories.csv", dtype = {"document_id":int, "category_id":int, "confidence_level":float})
    
    #rename columns of repeating names
    documents_topics.rename(columns={'confidence_level': 'confi_top'}, inplace=True)
    documents_categories.rename(columns={'confidence_level': 'confi_cat'}, inplace=True)
    
    #get only highest confidence topic and category
    max_rows = documents_categories.groupby(['document_id'])['confi_cat'].transform(max) == documents_categories['confi_cat']
    documents_categories = documents_categories[max_rows].drop_duplicates(subset = ['document_id','confi_cat'],keep = 'last')
    max_rows = documents_topics.groupby(['document_id'])['confi_top'].transform(max) == documents_topics['confi_top']
    documents_topics = documents_topics[max_rows].drop_duplicates(subset = ['document_id','confi_top'],keep = 'last')

    #create one table for both categories and topics
    topics_categories = documents_topics.merge(documents_categories, how = 'outer', on = 'document_id')
    del max_rows, documents_categories, documents_topics
    
    #assign nulls to -1 and cast the ids back to int (the merge changes them to float)
    topics_categories[['confi_cat', 'confi_top']] = topics_categories[['confi_cat', 'confi_top']].fillna(0)
    topics_categories = topics_categories.fillna(-1)
    topics_categories[['document_id', 'topic_id', 'category_id']] = topics_categories[['document_id', 'topic_id', 'category_id']].astype(int)
    
    topics_categories.to_csv(path_b + 'topics_categories.csv', index=False)
    print "topics_categories.csv created in path/source_tables/built/ directory"
    

if prep_promoted_content:
    promoted = p.read_csv(path_t + "promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
    if not prep_topics_categories:
        topics_categories = p.read_csv(path_b + "topics_categories.csv", dtype={"document_id":int, "topic_id":int, "confi_top":float, "category_id":int, "confi_cat":float})
    #create promoted, promoted content table (ad information) with its' categories and topics
    promoted = promoted.merge(topics_categories, how='left', on='document_id')
    promoted.rename(columns={'document_id': 'ad_document_id'}, inplace=True)
    
    #assign nulls to -1 and cast the ids back to int (the merge changes them to float)
    promoted[['confi_cat', 'confi_top']] = promoted[['confi_cat', 'confi_top']].fillna(0)
    promoted = promoted.fillna(-1)
    promoted[['topic_id', 'category_id']] = promoted[['topic_id', 'category_id']].astype(int)
    
    promoted.to_csv(path_b + 'promoted_content_prep.csv', index=False)
    print "promoted_content_prep.csv created in path/source_tables/built/ directory"

promoted_content_prep.csv created in path/source_tables/built/ directory
